# Agent 2: AI Agent with memory


In [2]:
import os
from typing import TypedDict, List, Union

from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
class AgentState(TypedDict):
    # -> the message can be of type HumanMessage or AIMessage
    msg: List[Union[HumanMessage, AIMessage]]

In [ ]:
llm = ChatGroq(model="openai/gpt-oss-120b")

In [ ]:
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["msg"])

    state["msg"].append(AIMessage(content=response.content))

    print(f"\nAI: {response.content}")
    # print("CURRENT STATE: ", state["msg"])
    return state

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

In [ ]:
conv_history = []  # -> NEW!!!

In [ ]:
user_input = input("enter: ")

while user_input != "exit":
    # add user input to history
    conv_history.append(HumanMessage(content=user_input))

    result = agent.invoke({
        "msg": conv_history  # -> send the whole history to the llm NOT JUST THE LAST INPUT
    })

    print(result['msg'])
    conv_history = result['msg']

    user_input = input("enter: ")


AI: Hello! How can I help you today?
[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

AI: Nice to meet you, Youssef! How can I assist you today?
[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='my name is youssef', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Youssef! How can I assist you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

AI: Your name is Youssef.
[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_

In [18]:
with open("logging.txt", "w", encoding="utf-8") as file:
    file.write("Your Conversation Log:\n")
    file.write("="*50 + "\n\n")

    for message in conv_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")

    file.write("="*50)
    file.write("\nEnd of Conversation\n")
    file.write("="*50 + '\n\n')

print("Conversation saved to logging.txt")

Conversation saved to logging.txt
